# AMORE
### How to evaluate models using last hidden state

In [2]:
from eval_faiss import *
import pandas as pd

from transformers import AutoTokenizer, AutoModel

C:\Users\veron\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First import libs and ```eval_faiss.py```. Then upload data and put it into ```full_data``` dict. Use augmentations that you need, you can evaluate models on all of them or on the choosed ones. If you prefer to use your own augmentations, also add them here.

In [3]:
canonical = pd.read_csv('./test_data/test_rdkit_canonical.csv', sep='\t')
kekulize = pd.read_csv('./test_data/test_kekulize_smiles.csv', sep='\t')
explicit_hs = pd.read_csv('./test_data/test_explicit_hs.csv', sep='\t')
cycle = pd.read_csv('./test_data/test_cycle_renumering.csv', sep='\t')
gold = pd.read_csv('./test_data/test_original.csv', sep='\t')

full_data = {'canonical': canonical, 'explicit_hs': explicit_hs, 'kekulize': kekulize, 'cycle': cycle}

In [4]:
"""
gold = pd.read_csv('./datasets/isomers/smiles_qm9_isomers_found_new.csv', sep='\t')

canonical = pd.read_csv('./datasets/isomers/isomers_rdkit_canonical.csv', sep='\t')
explicit_hs = pd.read_csv('./datasets/isomers/isomers_explicit_hs.csv', sep='\t')
kekulize = pd.read_csv('./datasets/isomers/isomers_kekulize.csv', sep='\t')
cycle = pd.read_csv('./datasets/isomers/isomers_cycle.csv', sep='\t')


full_data = {'canonical': canonical, 'explicit_hs': explicit_hs, 'kekulize': kekulize, 'cycle': cycle}
"""

"\ngold = pd.read_csv('./datasets/isomers/smiles_qm9_isomers_found_new.csv', sep='\t')\n\ncanonical = pd.read_csv('./datasets/isomers/isomers_rdkit_canonical.csv', sep='\t')\nexplicit_hs = pd.read_csv('./datasets/isomers/isomers_explicit_hs.csv', sep='\t')\nkekulize = pd.read_csv('./datasets/isomers/isomers_kekulize.csv', sep='\t')\ncycle = pd.read_csv('./datasets/isomers/isomers_cycle.csv', sep='\t')\n\n\nfull_data = {'canonical': canonical, 'explicit_hs': explicit_hs, 'kekulize': kekulize, 'cycle': cycle}\n"

Here is the main function of evaluation, it will get last hidden state from encoder or will use last hidden state of bert-like models

In [5]:
def test_model(model, tokenizer, full_data=full_data):
    
    try:
        orig_res = save_last_hidden_state_encoder(gold, model, tokenizer, 'orig')
    except:
        print('model mode')
        orig_res = save_last_hidden_state_model(gold, model, tokenizer, 'orig')

    for key, value in full_data.items():
        try:
            test = save_last_hidden_state_encoder(value, model, tokenizer, key)
        except:
            print('model mode')
            test = save_last_hidden_state_model(value, model, tokenizer, key)
        x = create_dist_MRR(test, orig_res)
        print(key)
        print('MRR')
        print(sum([1/(i+1) for i in x[-1]])/len(x[-1]))

In [5]:
tokenizer = AutoTokenizer.from_pretrained("laituan245/molt5-base-smiles2caption")
model = AutoModel.from_pretrained('laituan245/molt5-base-smiles2caption')
test_model(model, tokenizer)

100%|██████████| 3300/3300 [04:47<00:00, 11.47it/s]


canonical
MRR
0.5090194203470724


100%|██████████| 3300/3300 [08:36<00:00,  6.39it/s]


explicit_hs
MRR
0.05474498500082556


100%|██████████| 3300/3300 [04:57<00:00, 11.10it/s]


kekulize
MRR
0.7086395625615578


100%|██████████| 3300/3300 [05:00<00:00, 10.99it/s]


cycle
MRR
0.9382382746514041


In [6]:
tokenizer = AutoTokenizer.from_pretrained("GT4SD/multitask-text-and-chemistry-t5-base-standard")
model = AutoModel.from_pretrained('GT4SD/multitask-text-and-chemistry-t5-base-standard')
test_model(model, tokenizer)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
100%|██████████| 3300/3300 [04:05<00:00, 13.46it/s]


canonical
MRR
0.6308568066328277


100%|██████████| 3300/3300 [07:02<00:00,  7.81it/s]


explicit_hs
MRR
0.0841488879516965


100%|██████████| 3300/3300 [04:07<00:00, 13.36it/s]


kekulize
MRR
0.7782348126385085


100%|██████████| 3300/3300 [04:09<00:00, 13.20it/s]


cycle
MRR
0.9842264309764309


In [9]:
tokenizer = AutoTokenizer.from_pretrained('GT4SD/multitask-text-and-chemistry-t5-base-augm')
model = AutoModel.from_pretrained('GT4SD/multitask-text-and-chemistry-t5-base-augm')
test_model(model, tokenizer)

100%|██████████| 3300/3300 [04:52<00:00, 11.29it/s]


In [7]:
tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
model = AutoModel.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')
test_model(model, tokenizer)

  0%|          | 0/3300 [00:00<?, ?it/s]


model mode


  0%|          | 0/3300 [00:00<?, ?it/s]


model mode


100%|██████████| 3300/3300 [01:51<00:00, 29.65it/s]


...

kekulize
MRR
0.5412084946588809


  0%|          | 0/3300 [00:00<?, ?it/s]


model mode


100%|██████████| 3300/3300 [01:58<00:00, 27.85it/s]


cycle
MRR
0.9525582814522532


In [6]:
tokenizer = AutoTokenizer.from_pretrained("mschuh/PubChemDeBERTa-augmented")
model = AutoModel.from_pretrained('mschuh/PubChemDeBERTa-augmented')
test_model(model, tokenizer)

  0%|          | 0/3300 [00:00<?, ?it/s]


model mode


  0%|          | 0/3300 [00:00<?, ?it/s]


model mode


100%|██████████| 3300/3300 [10:32<00:00,  5.22it/s]


...

kekulize
MRR
0.628572934778063


  0%|          | 0/3300 [00:00<?, ?it/s]


model mode


100%|██████████| 3300/3300 [11:40<00:00,  4.71it/s]


cycle
MRR
0.9785800347092214


In [16]:
tokenizer = AutoTokenizer.from_pretrained("entropy/roberta_zinc_480m", max_len=128)# размерности тензоров не совпадают
model = AutoModel.from_pretrained('entropy/roberta_zinc_480m')

orig_res = save_last_hidden_state_bert(gold['SMILES'], model, tokenizer, 'orig')

for key, value in full_data.items():
    test = save_last_hidden_state_bert(value['SMILES'], model, tokenizer, key)
    #x = create_dist(test, orig_res)
    x = create_dist_MRR(test, orig_res)
    print(key)
    print('MRR')
    print(sum([1/(i+1) for i in x[-1]])/len(x[-1]))


Some weights of RobertaModel were not initialized from the model checkpoint at entropy/roberta_zinc_480m and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 3300/3300 [06:56<00:00,  7.93it/s]


canonical
MRR
0.3319206577158926


100%|██████████| 3300/3300 [08:09<00:00,  6.74it/s]


explicit_hs
MRR
0.01690927781786464


100%|██████████| 3300/3300 [07:04<00:00,  7.77it/s]


kekulize
MRR
0.45584311454706455


100%|██████████| 3300/3300 [07:07<00:00,  7.73it/s]


cycle
MRR
0.6918379029800314


In [8]:
tokenizer = AutoTokenizer.from_pretrained("laituan245/molt5-large-smiles2caption")
model = AutoModel.from_pretrained('laituan245/molt5-large-smiles2caption')
test_model(model, tokenizer)

100%|██████████| 3300/3300 [14:35<00:00,  3.77it/s]


canonical
MRR
0.5473938868323259


100%|██████████| 3300/3300 [26:21<00:00,  2.09it/s]


explicit_hs
MRR
0.04149201506590143


100%|██████████| 3300/3300 [14:40<00:00,  3.75it/s]


kekulize
MRR
0.6719233680723551


100%|██████████| 3300/3300 [16:24<00:00,  3.35it/s]


cycle
MRR
0.9906161616161616


In [9]:
tokenizer = AutoTokenizer.from_pretrained("entropy/gpt2_zinc_87m", max_len=256) # ids out of range 
model = AutoModel.from_pretrained('entropy/gpt2_zinc_87m')
test_model(model, tokenizer)

  0%|          | 0/3300 [00:00<?, ?it/s]


model mode


  0%|          | 0/3300 [00:00<?, ?it/s]


model mode


100%|██████████| 3300/3300 [04:22<00:00, 12.57it/s]


...

kekulize
MRR
0.41692750948809054


  0%|          | 0/3300 [00:00<?, ?it/s]


model mode


100%|██████████| 3300/3300 [04:43<00:00, 11.62it/s]


cycle
MRR
0.8001843020469118


In [10]:
tokenizer = AutoTokenizer.from_pretrained("jonghyunlee/ChemBERT_ChEMBL_pretrained", max_len=128) #256 max_length=128
model = AutoModel.from_pretrained('jonghyunlee/ChemBERT_ChEMBL_pretrained')

orig_res = save_last_hidden_state_bert(gold['SMILES'], model, tokenizer, 'orig')

for key, value in full_data.items():
    test = save_last_hidden_state_bert(value['SMILES'], model, tokenizer, key)
    #x = create_dist(test, orig_res)
    x = create_dist_MRR(test, orig_res)
    print(key)
    print('MRR')
    print(sum([1/(i+1) for i in x[-1]])/len(x[-1]))

Some weights of BertModel were not initialized from the model checkpoint at jonghyunlee/ChemBERT_ChEMBL_pretrained and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 3300/3300 [01:02<00:00, 53.03it/s]


canonical
MRR
0.3222900363445153


100%|██████████| 3300/3300 [01:20<00:00, 41.14it/s]


explicit_hs
MRR
0.028572756379835283


100%|██████████| 3300/3300 [01:02<00:00, 52.56it/s]


kekulize
MRR
0.4610637613875959


100%|██████████| 3300/3300 [01:03<00:00, 51.74it/s]


cycle
MRR
0.8320478471458127


In [11]:
tokenizer = AutoTokenizer.from_pretrained("gayane/BARTSmiles", trust_remote_code=True, max_len=128)
model = AutoModel.from_pretrained("gayane/BARTSmiles", trust_remote_code=True)

orig_res = save_last_hidden_state_bert(gold['SMILES'], model, tokenizer, 'orig')

for key, value in full_data.items():
    test = save_last_hidden_state_bert(value['SMILES'], model, tokenizer, key)
    x = create_dist_MRR(test, orig_res)
    print(key)
    print('MRR')
    print(sum([1/(i+1) for i in x[-1]])/len(x[-1]))

100%|██████████| 3300/3300 [13:59<00:00,  3.93it/s]


canonical
MRR
0.31844008784732564


100%|██████████| 3300/3300 [24:19<00:00,  2.26it/s]


explicit_hs
MRR
0.021607089132251917


100%|██████████| 3300/3300 [14:20<00:00,  3.84it/s]


kekulize
MRR
0.46937777487148913


100%|██████████| 3300/3300 [14:40<00:00,  3.75it/s]


cycle
MRR
0.662280967138691


In [12]:
def save_last_hidden_state_t5(data, model, tokenizer, path_to_res): 
    model.eval()
    with torch.no_grad():
        t5_condgen_result = []
        for batch in tqdm(data):
            
            inputs = tokenizer(batch, return_tensors="pt", return_token_type_ids=False, padding=False, truncation=True)
            output = model.encoder(input_ids=inputs, return_dict=True).last_hidden_state
            t5_condgen_result.append(torch.mean(output, dim=1))

    with open(path_to_res + '_hidden.pkl', 'wb') as f:
        pickle.dump(t5_condgen_result, f)

In [13]:
tokenizer = AutoTokenizer.from_pretrained("GT4SD/multitask-text-and-chemistry-t5-base-standard")
model = AutoModel.from_pretrained('GT4SD/multitask-text-and-chemistry-t5-base-standard')
model.eval()

gold['SMILES'] = ['Caption the following SMILES: '+i for i in gold['SMILES']]
orig_res = save_last_hidden_state_encoder(gold, model, tokenizer, 'orig')

for key, value in full_data.items():
    value['SMILES'] = ['Caption the following SMILES: '+i for i in value['SMILES']]
    test = save_last_hidden_state_encoder(value, model, tokenizer, key)
    #x = create_dist(test, orig_res)
    x = create_dist_MRR(test, orig_res)
    print(key)
    print('MRR')
    print(sum([1/(i+1) for i in x[-1]])/len(x[-1]))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
100%|██████████| 3300/3300 [04:45<00:00, 11.56it/s]


canonical
MRR
0.72413124747378


100%|██████████| 3300/3300 [08:01<00:00,  6.85it/s]


explicit_hs
MRR
0.08569781859059095


100%|██████████| 3300/3300 [04:43<00:00, 11.62it/s]


kekulize
MRR
0.8378026761797364


100%|██████████| 3300/3300 [04:49<00:00, 11.40it/s]


cycle
MRR
0.9817955781592146


In [14]:
tokenizer = AutoTokenizer.from_pretrained("GT4SD/multitask-text-and-chemistry-t5-base-augm")
model = AutoModel.from_pretrained('GT4SD/multitask-text-and-chemistry-t5-base-augm')
model.eval()

gold['SMILES'] = ['Caption the following SMILES: '+i for i in gold['SMILES']]
orig_res = save_last_hidden_state_encoder(gold, model, tokenizer, 'orig')

for key, value in full_data.items():
    value['SMILES'] = ['Caption the following SMILES: '+i for i in value['SMILES']]
    test = save_last_hidden_state_encoder(value, model, tokenizer, key)
    #x = create_dist(test, orig_res)
    x = create_dist_MRR(test, orig_res)
    print(key)
    print('MRR')
    print(sum([1/(i+1) for i in x[-1]])/len(x[-1]))

100%|██████████| 3300/3300 [05:10<00:00, 10.63it/s]


canonical
MRR
0.708984162496612


100%|██████████| 3300/3300 [07:52<00:00,  6.98it/s]


explicit_hs
MRR
0.07115887116545133


100%|██████████| 3300/3300 [04:50<00:00, 11.37it/s]


kekulize
MRR
0.8438527616890797


100%|██████████| 3300/3300 [04:52<00:00, 11.28it/s]


cycle
MRR
0.9834855699855702


In [15]:
tokenizer = AutoTokenizer.from_pretrained("razent/SciFive-base-Pubmed")
model = AutoModel.from_pretrained("razent/SciFive-base-Pubmed")

orig_res = save_last_hidden_state_scifive(gold['SMILES'], model, tokenizer, 'orig')

for key, value in full_data.items():
    test = save_last_hidden_state_scifive(value['SMILES'], model, tokenizer, key)
    x = create_dist_MRR(test, orig_res)
    print(key)
    print('MRR')
    print(sum([1/(i+1) for i in x[-1]])/len(x[-1]))

100%|██████████| 3300/3300 [05:04<00:00, 10.85it/s]


canonical
MRR
0.3995886613287398


100%|██████████| 3300/3300 [08:13<00:00,  6.69it/s]


explicit_hs
MRR
0.02971949426139215


100%|██████████| 3300/3300 [04:58<00:00, 11.07it/s]


kekulize
MRR
0.6240719026867979


100%|██████████| 3300/3300 [05:01<00:00, 10.96it/s]


cycle
MRR
0.9922323232323234
